In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd

import tensorflow.compat.v1 as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
import keras
import matplotlib
import tensorflow_datasets as tfds

import shutil
from tqdm import tqdm

import warnings
import random
warnings.filterwarnings("ignore", category=UserWarning) #used to supress the tf version warning. 

mids_dir = Path("D:\\MIDS-W207")
data = mids_dir/"datasets"
project = mids_dir/"MIDS-W207-Spring24-Soccer-Detection"
analysis = project/"analysis"

# Author: Timothy Majidzadeh
# Date Created: March 31, 2024
# Date Updated: April 3, 2024
# Description: Create a square dataset from the top-view and wide-view datasets.
# Notes: [v1] Created program.
# Inputs: Frame-by-frame image data & labels.
# Outputs: 800x800 square images & associated bounding boxes.

In [2]:
top_view_height, top_view_width = 2160, 3840
wide_view_height, wide_view_width = 1000, 6500
target_height, target_width = 800, 800

In [3]:
labels_order = [
    ('vidname', '', ''),
    ('frame', '', ''),
    ('frame_imgname', '', ''),
    ('frame_imgpath', '', ''),
    ('frame_saved', '', ''),
    ('ball', 'ball', 'bb_left'),
    ('ball', 'ball', 'bb_top'),
    ('ball', 'ball', 'bb_width'),
    ('ball', 'ball', 'bb_height'),
    ('team_0', 'player_00', 'bb_left'),
    ('team_0', 'player_00', 'bb_top'),
    ('team_0', 'player_00', 'bb_width'),
    ('team_0', 'player_00', 'bb_height'),
    ('team_0', 'player_01', 'bb_left'),
    ('team_0', 'player_01', 'bb_top'),
    ('team_0', 'player_01', 'bb_width'),
    ('team_0', 'player_01', 'bb_height'),
    ('team_0', 'player_02', 'bb_left'),
    ('team_0', 'player_02', 'bb_top'),
    ('team_0', 'player_02', 'bb_width'),
    ('team_0', 'player_02', 'bb_height'),
    ('team_0', 'player_03', 'bb_left'),
    ('team_0', 'player_03', 'bb_top'),
    ('team_0', 'player_03', 'bb_width'),
    ('team_0', 'player_03', 'bb_height'),
    ('team_0', 'player_04', 'bb_left'),
    ('team_0', 'player_04', 'bb_top'),
    ('team_0', 'player_04', 'bb_width'),
    ('team_0', 'player_04', 'bb_height'),
    ('team_0', 'player_05', 'bb_left'),
    ('team_0', 'player_05', 'bb_top'),
    ('team_0', 'player_05', 'bb_width'),
    ('team_0', 'player_05', 'bb_height'),
    ('team_0', 'player_06', 'bb_left'),
    ('team_0', 'player_06', 'bb_top'),
    ('team_0', 'player_06', 'bb_width'),
    ('team_0', 'player_06', 'bb_height'),
    ('team_0', 'player_07', 'bb_left'),
    ('team_0', 'player_07', 'bb_top'),
    ('team_0', 'player_07', 'bb_width'),
    ('team_0', 'player_07', 'bb_height'),
    ('team_0', 'player_08', 'bb_left'),
    ('team_0', 'player_08', 'bb_top'),
    ('team_0', 'player_08', 'bb_width'),
    ('team_0', 'player_08', 'bb_height'),
    ('team_0', 'player_09', 'bb_left'),
    ('team_0', 'player_09', 'bb_top'),
    ('team_0', 'player_09', 'bb_width'),
    ('team_0', 'player_09', 'bb_height'),
    ('team_0', 'player_10', 'bb_left'),
    ('team_0', 'player_10', 'bb_top'),
    ('team_0', 'player_10', 'bb_width'),
    ('team_0', 'player_10', 'bb_height'),
    ('team_1', 'player_00', 'bb_left'),
    ('team_1', 'player_00', 'bb_top'),
    ('team_1', 'player_00', 'bb_width'),
    ('team_1', 'player_00', 'bb_height'),
    ('team_1', 'player_01', 'bb_left'),
    ('team_1', 'player_01', 'bb_top'),
    ('team_1', 'player_01', 'bb_width'),
    ('team_1', 'player_01', 'bb_height'),
    ('team_1', 'player_02', 'bb_left'),
    ('team_1', 'player_02', 'bb_top'),
    ('team_1', 'player_02', 'bb_width'),
    ('team_1', 'player_02', 'bb_height'),
    ('team_1', 'player_03', 'bb_left'),
    ('team_1', 'player_03', 'bb_top'),
    ('team_1', 'player_03', 'bb_width'),
    ('team_1', 'player_03', 'bb_height'),
    ('team_1', 'player_04', 'bb_left'),
    ('team_1', 'player_04', 'bb_top'),
    ('team_1', 'player_04', 'bb_width'),
    ('team_1', 'player_04', 'bb_height'),
    ('team_1', 'player_05', 'bb_left'),
    ('team_1', 'player_05', 'bb_top'),
    ('team_1', 'player_05', 'bb_width'),
    ('team_1', 'player_05', 'bb_height'),
    ('team_1', 'player_06', 'bb_left'),
    ('team_1', 'player_06', 'bb_top'),
    ('team_1', 'player_06', 'bb_width'),
    ('team_1', 'player_06', 'bb_height'),
    ('team_1', 'player_07', 'bb_left'),
    ('team_1', 'player_07', 'bb_top'),
    ('team_1', 'player_07', 'bb_width'),
    ('team_1', 'player_07', 'bb_height'),
    ('team_1', 'player_08', 'bb_left'),
    ('team_1', 'player_08', 'bb_top'),
    ('team_1', 'player_08', 'bb_width'),
    ('team_1', 'player_08', 'bb_height'),
    ('team_1', 'player_09', 'bb_left'),
    ('team_1', 'player_09', 'bb_top'),
    ('team_1', 'player_09', 'bb_width'),
    ('team_1', 'player_09', 'bb_height'),
    ('team_1', 'player_10', 'bb_left'),
    ('team_1', 'player_10', 'bb_top'),
    ('team_1', 'player_10', 'bb_width'),
    ('team_1', 'player_10', 'bb_height')
]

class_dict = {
    'ball':0,
    'team_0':1,
    'team_1':2
}

In [4]:
def preprocess_labels(input_df):
    '''
    Input dataframe is in wide format with multilevel columns.
    Reshape to long format and generate the class value.
    Inputs:
        input_df: The stacked labels from the soccertrack dataset, in a wide Pandas DataFrame.
    Outputs:
        output_df: The labels reshaped to wide and with necessary values generated.
    '''
    output_df = input_df[labels_order]
    output_df = output_df.reorder_levels([2, 0, 1], axis=1)
    output_df.columns = ['-'.join(col).strip() for col in output_df.columns.values]
    output_df.rename(columns={"-vidname-":"vidname", "-frame-":"frame", "-frame_imgname-":"frame_imgname", "-frame_imgpath-":"frame_imgpath", "-frame_saved-":"frame_saved"}
                       ,inplace=True)
    i = ['vidname', 'frame', 'frame_imgname', 'frame_imgpath', 'frame_saved']
    output_df = pd.wide_to_long(
        output_df,
        stubnames=['bb_left', 'bb_top', 'bb_width', 'bb_height'], 
        i=i, 
        j='player', 
        suffix=r'.*').reset_index()
    output_df['player'] = output_df['player'].str.replace('^-', '', regex=True)
    output_df[['team', 'player']] = output_df['player'].str.split('-', n=1, expand=True)
    output_df['class'] = output_df['team'].map(class_dict)
    output_df['bb_right'] = output_df['bb_left'] + output_df['bb_width']
    output_df['bb_bot'] = output_df['bb_top'] + output_df['bb_height']
    output_df['bb_xcenter'] = output_df['bb_left'] + output_df['bb_width'] / 2
    output_df['bb_ycenter'] = output_df['bb_top'] + output_df['bb_height'] / 2
    
    output_df = output_df[['vidname', 'frame', 'frame_imgname', 'frame_imgpath', 'frame_saved', 'player', 'team', 'class', 'bb_left', 'bb_top', 'bb_right', 'bb_bot', 'bb_xcenter', 'bb_ycenter', 'bb_width', 'bb_height']]
    return(output_df)

In [7]:
top_view_labels = pd.read_pickle(data/"soccertrack/labels/top_view_labels_stacked/top_view_labels.pkl")
top_view_labels_long = preprocess_labels(top_view_labels[top_view_labels['frame_saved'] == True])
paths = top_view_labels_long['frame_imgpath'].unique()

In [20]:
top_view_labels_long[top_view_labels_long['frame_imgpath'] == paths[31045]]

,vidname,frame,frame_imgname,frame_imgpath,frame_saved,player,team,class,bb_left,bb_top,bb_right,bb_bot,bb_xcenter,bb_ycenter,bb_width,bb_height
714035,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,ball,ball,0,1886.0,587.0,1898.0,599.0,1892.0,593.0,12.0,12.0
714036,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_00,team_0,1,898.0,1038.0,930.0,1060.0,914.0,1049.0,32.0,22.0
714037,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_01,team_0,1,1586.0,1114.0,1610.0,1138.0,1598.0,1126.0,24.0,24.0
714038,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_02,team_0,1,2104.0,950.0,2130.0,974.0,2117.0,962.0,26.0,24.0
714039,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_03,team_0,1,1558.0,1390.0,1582.0,1420.0,1570.0,1405.0,24.0,30.0
714040,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_04,team_0,1,1992.0,1384.0,2016.0,1406.0,2004.0,1395.0,24.0,22.0
714041,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_05,team_0,1,1818.0,590.0,1846.0,614.0,1832.0,602.0,28.0,24.0
714042,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_06,team_0,1,1856.0,366.0,1884.0,386.0,1870.0,376.0,28.0,20.0
714043,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_07,team_0,1,1726.0,1060.0,1750.0,1084.0,1738.0,1072.0,24.0,24.0
714044,D_20220220_1_1050_1080,1,D_20220220_1_1050_1080_1.png,D:\MIDS-W207\datasets\soccertrack/images/top_v...,True,player_08,team_0,1,1514.0,798.0,1538.0,818.0,1526.0,808.0,24.0,20.0
